# Tasks

## Task 1: Predict the topic labels of discussions

### Purpose

We apply the natural language technologies, such as n-gram and stemming, we've learned in class to generate useful features and evaluate their effectiveness on classifying argument discussions into 10 labeled topic groups. The architecture for this task is as below:

![alt text](imgs/task1-arch.png "Architecture")

According to our observation, people often use different keywords when discussing different topics. We develop a keyword-based approach to classify dicussions into topics. The intuition is to identify keywords that are not frequently used for each topic, so we can classify a discussion based on the keywords used in it.

### Preprocessing

For each discussion, 
1. We filtered characters that are neither numbers nor english letters (e.g. "[A-Za-z0-9]" in regular expression). 
2. We filtered stopwords in text. The stopword list, which contains 2,400 stopwords for 11 languages, is from the stopword corpus created by Porter et al. (2001). 
3. We adopted Porter stemmer to convert words in dicussions to their stemmed form. There are 98,988 distinct stems used in our dataset.
4. We took the stems as unigrams and built an unigram occurrence vector as the representation of discussion.

Since the number of unigrams is large, most of representation vectors are sparse. It is an known issue which may affect classification performance. Our future goal would be adopt some dimension reduction methods to reduce the size of feature vectors.

### Topic Classification


| Topic        | # of discussions  |
| ------------- |:-------------:|
| abortion | 564 |
| climate change | 40 |
| communism vs capitalism | 38 |
| death penalty | 25 |
| evolution | 871 |
| existence of God | 105 |
| gay marriage | 305 |
| gun control | 824 |
| healthcare | 81 |
| marijuana legalization | 13 |

The above figure shows the distribution of topics. We split the dataset into 75% stratified training data and 25% testing data. We trained a multi-class SVM model with linear kernel (LIBSVM, proposed by Chang et al., 2011). The experiment was conducted in 5-fold cross validation. The following are the accuaracy values of cross validation:

| 1 round accuracy  | 2 round accuracy  | 3 round accuracy  | 4 round accuracy  | 5 round accuracy  | Mean accuracy |
| ------------- |:-------------|:-------------|:-------------|:-------------|:-------------:|
| 0.885416666667 | 0.883275261324 | 0.858885017422 | 0.867132867133 | 0.884210526316 | 0.8757840677721935 |


The average accuracy of our result is 87.58%, which indicates the unigram occurrance representation is significantly effective in this task. Keyword-based representation performs well to recognize the topic of discussions.



## Task 2: Identify author stance in discussions with Doc2Vec

### Purpose


In a discussion, authors express their different stances toward a topic. We want to discover how to effectively identify an author's stance in a discussion based on his posts. Since the IAC dataset contains the annotations of author stance for discussions, we try to convert author posts in discussion to Doc2Vec (Le and Mikolov, 2014) representation and evaluate if the representation is effective on identifying user stance.

In our analysis, we peformed both clustering and classification to evaluate the performance of the Doc2Vec features.

### Preprocessing

For each discussion, we first identify the authors. For each author, we first filtered characters that are neither numbers nor english letters (e.g. "[A-Za-z0-9]" in regular expression) in his posts. All filtered posts were concatenated and converted to a Doc2Vec vector.

In IAC dataset, the author stance on a discussion is recorded as a combination of votes from annotators. Each annotator can vote "pro", "anti", or "other". We use the following rules to consider the majority vote as the label of author stance:
- If the number of "pro" votes equals to "anti", or "other" dominates, choose "other" as the stance label.
- If "pro" dominates, choose "pro" as the stance label.
- If "anti" dominates, choose "anti" as the stance label.

### Analysis by Kmeans clustering

We perform K-means clustering (Kanungo et al., 2002) with K=3 for every topic respectively. Our metrics is Adjusted Mutual Infomation Score (Vinh et al., 2010). The value of AMI is used to evaluate the extent of shared information between two clustering results. Below is the results of different topics:

| Topic | Adjusted Mutual Infomation Score (AMI) |
| ------------- |:-------------:|
| gay marriage | 0.00698645511313198 |
| existence of God | 0.016446639162490496 |
| death penalty | 0.008976145629304096 |
| abortion | 0.00410753125937423 |
| climate change | -0.0002261692372829449 |
| healthcare | 0.005124345023058839 |
| evolution | 0.08299409157326207 |
| communism vs capitalism | 0.04538428687976163 |
| gun control | 0.0029354720744514434 |
| marijuana legalization | 0.06779748379608597 |

According to the results, using Doc2Vec representation stand-alone is not effective on identifying author stance in discussions.


### Analysis by SVM classification

We follow the same settings in task 1 to train a three-class SVM model. The following are the accuracy values of 5-fold cross validation in different topics:

| Topic        | 1 round accuracy  | 2 round accuracy  | 3 round accuracy  | 4 round accuracy  | 5 round accuracy  | Mean accuracy |
| ------------- |:-------------|:-------------|:-------------|:-------------|:-------------|:-------------:|
| communism vs capitalism | 0.6 | 0.7925925926 | 0.5671641791 | 0.6390977444 | 0.5454545455 | 0.6288618123 |
| marijuana legalization | 0.6465517241 | 0.6637931034 | 0.7543859649 | 0.7876106195 | 0.7256637168 | 0.7156010258 |
| death penalty | 0.4538745387 | 0.5444444444 | 0.4721189591 | 0.4646840149 | 0.4589552239 | 0.4788154362 |
| climate change | 0.4829931973 | 0.4794520548 | 0.4965517241 | 0.4206896552 | 0.3333333333 | 0.4426039929 |
| existence of God | 0.4344163658 | 0.4843373494 | 0.4542168675 | 0.4909529554 | 0.4685990338 | 0.4665045144 |
| healthcare | 0.6 | 0.4428571429 | 0.3857142857 | 0.6231884058 | 0.5735294118 | 0.5250578492 |
| gun control | 0.642765685 | 0.4628205128 | 0.5217948718 | 0.5481386393 | 0.5468549422 | 0.5444749302 |
| abortion | 0.4704433498 | 0.4568965517 | 0.5289765721 | 0.4358816276 | 0.5147965475 | 0.4813989297 |
| gay marriage | 0.6481876333 | 0.5896159317 | 0.6324786325 | 0.5861823362 | 0.5961538462 | 0.610523676 |
| evolution | 0.598546042 | 0.5331179321 | 0.598546042 | 0.5828617623 | 0.6437246964 | 0.591359295 |

The classification results show the Doc2Vec representation only performs  in most of topics.